# Introduktion til `spaCy` 

"spaCy" indeholder forskellige sprogmodeller - herunder en dansk sprogmodel.

Overordnet virker spaCy ved, at man specificerer en sprogmodel samt nogen "processors", som modellen skal indeholde. 

SpaCy's sprogmodeller indeholder blandt andet:
- Tokenizer (inddeling i enkeltord)
- Lemmatizer (konvertering til navneform)
- Part-Of-Speech tagging (POS-tagging) (identificering af ordtyper)
- Dependency parsing (sætningskonstruktion)
- Named-Entity-Recognition (NER) (udledning af "named entities", fx personer og organisationer)

## Brug af spaCy i Python

1. Indlæs sprogmodel
2. Analysér tekstykke
3. Inspicér resultater

In [1]:
import spacy

#!python -m spacy download 'da_core_news_sm' # evt. installer sprogmodel

Når sprogmodellen er hentet, kan vi bruge den ved at indlæse modellen. Som standard indlæses modellen med alle processerne, men det er muligt at aktivere/deaktivere specifikke processer.

Efter modellen er defineret, kan man lade sprogmodellen analysere tekst.

In [2]:
nlp = spacy.load("da_core_news_sm") # Definerer model

doc = nlp('Politiet har givet borgerne råd') # Analyserer tekst med model

Når modellen anvendes på et stykke tekst, behandler den tekststykket med de forskellige processors, som er en del af sprogmodellen (som standard for dansk: tokenizer, part-of-speech tagging, lemmatizer og dependency parsing).

Outputtet (`doc`) indeholder de forskellige værdier, som er udledt af teksten, som attributes (et attribute for token, et for lemma, et for POS-tag osv.).

Vi kan fx visualisere sætningskonstruktionen med funktionen `displacy`:

In [3]:
from spacy import displacy # skal indlæses separat

displacy.render(doc, style='dep')

## Lemmatizing

Et ords "lemma" er dets grammatiske stamme (fx "er"->"være", "spiste"->"spise"). SpaCy's sprogmodeller indeholder typisk en indbygget ordbog til at finde stammen for de enkelte ord. Et ords "lemma" er gem under attributtet `.lemma_` for hvert ord:

In [4]:
for word in doc:
    print(f'{word.text:<15} {word.lemma_}')

Politiet        politi
har             have
givet           give
borgerne        borger
råd             råd


## Part-of-speech tags

SpaCy tagger automatisk hvert ord med sin ordklasse ("part-of-speech"-tag/POS-tag). Disse er gemt under attributtet `.pos_` for hvert ord:

In [5]:
for word in doc:
    print(f'{word.text:<15} {word.pos_}')

Politiet        NOUN
har             AUX
givet           VERB
borgerne        NOUN
råd             NOUN


Part-of-speech tagging virker ved, at modellen i forvejen er trænet på danske tekster, og derfor har "set" de forskellige ord i kontekst før. Som det kan ses, er modellen dog ikke perfekt (fx "trygge" er angivet som navneord (NOUN), selvom der her er tale om et tillægsord (ADJ)).

Part-of-speech tagging tillader fx at isolere visse ord i et stykke tekst:

In [6]:
keep_tags = ['NOUN', 'ADJ', 'PROPN']
keep_words = []

for word in doc:
    if word.pos_ in keep_tags:
        keep_words.append(word)

for word in keep_words:
    print(f'{word.text:<15} {word.pos_}')

Politiet        NOUN
borgerne        NOUN
råd             NOUN


## Dependency parsing

SpaCy laver også analyse af sætningskonstruktion (dependency parsing). Hvilken del af sætningen, som ordene er analyseret frem til, kan tilgås af attribut `.dep_`.

In [7]:
for word in doc:
    print(f'{word.text:<15} {word.dep_}')

Politiet        nsubj
har             aux
givet           ROOT
borgerne        iobj
råd             obj


## Named entities

"Named entities" kan groft sagt forstås som "meningsfulde enheder" i teksten. Det kan fx være personer, organisationer eller steder. Ligesom ved part-of-speech tagging, fungerer "named entity recognition" ved, at modellen enten har set disse enheder før eller er bekendt med, hvordan sådanne enheder fremgår i sætningen (hvor er de i sætningskonstruktionen, hvilke ordklasser er de associeret med).

Alle ord i en tekst er ikke en "named entity". Named entities kan tilgås gennem attributtet `ents` for det behandlede stykke tekst (`doc`). Fra dette kan ses, hvilke enheder er udledt, og hvordan de er kategoriseret:

In [8]:
doc = nlp("Søs Marie Serup, politisk kommentator og tidligere særlig rådgiver for Løkke, fortalte i fredags i DR's nyhedspodcast 'Genstart' om hans evne til altid at komme tilbage i politik.")

for ent in doc.ents:
    print(f'{ent.text:<15} {ent.label_}')

Marie Serup     PER
Løkke           LOC
DR's            ORG


Denne sprogmodel arbejder med fire named entity tags:
- LOC: Steder
- ORG: Organisationer
- PER: Personer
- MISC: Andet

Af ovenstående ses, at modellen identificerer "DR" som en organisation, hvilket er meget passende. Derudover genkender den "Marie Serup" som person, men har udeladt fornavnet "Søs". Dog er "Løkke" fejlklassificeret som et sted (LOC).

## Tilpas spaCy pipeline

Når man definerer `nlp`-funktionen (sit spaCy pipeline) med en sprogmodel (`spacy.load()`), inkluderes alle komponenter som standard. Hvis man kun er interesseret i specifikke komponenter, kan man slå dele af pipeline til eller fra. 

Se de forskellige komponenter her: [https://spacy.io/usage/processing-pipelines#built-in](https://spacy.io/usage/processing-pipelines#built-in).

Når man arbejder med store mængder tekstdata, kan det give mening at forsimple funktionen for at spare beregningstid.

### Slå komponenter fra

Man slår komponenter fra ved brug af argumentet `disable`, når man indlæser modellen.

In [9]:
nlp = spacy.load("da_core_news_sm", disable = ['parser'])

doc = nlp('Politiet har givet borgerne råd') # analysér tekststykke med nyt pipeline

`doc` indeholder stadig lemma, da lemmatizer stadig er slået til.

In [10]:
for word in doc:
    print(f'{word.text:<15} {word.lemma_}')

Politiet        politi
har             have
givet           give
borgerne        borger
råd             råd


Der er ikke længere nogen dependency labels, da `parser` er slået fra (returnerer None).

In [11]:
for word in doc:
    print(f'{word.text:<15} {word.dep_}')

Politiet        
har             
givet           
borgerne        
råd             


### Slå komponenter til

Man slår komponenter til ved brug af argumentet `enable`, når man indlæser modellen. Alle komponenter, som ikke listes, slås fra.

In [12]:
nlp = spacy.load("da_core_news_sm", enable = ['parser'])

doc = nlp('Politiet har givet borgerne råd') # analysér tekststykke med nyt pipeline

`doc` indeholder nu ikke lemma, da kun parser er slået til.

In [13]:
for word in doc:
    print(f'{word.text:<15} {word.lemma_}')

Politiet        
har             
givet           
borgerne        
råd             


Der er dependency labels, da parser er slået til.

In [14]:
for word in doc:
    print(f'{word.text:<15} {word.dep_}')

Politiet        ROOT
har             nsubj
givet           punct
borgerne        ROOT
råd             ROOT


### Tokenizer som særskilt funktion

Hvis man blot vil bruge tokenizeren, kan denne tilgås direkte.

In [15]:
nlp = spacy.load("da_core_news_sm")

tokenizer = nlp.tokenizer

In [16]:
tokenizer('Politiet har givet borgerne råd')

Politiet har givet borgerne råd

Tokenizeren returnerer stadig et `doc` objekt, men da den kun giver tokens tilbage, kan man tvinge output om til en liste:

In [17]:
list(tokenizer('Politiet har givet borgerne råd'))

[Politiet, har, givet, borgerne, råd]

## spaCy pipeline på flere stykker tekst

Pipeline-funktionen (`nlp`) virker kun på ét stykke tekst. Afhængig af datastruktur, kan man anvende pipeline på flere stykker tekst.

In [18]:
texts = [
    'Smileyordningen får stor makeover: Kontrolrapporten forsvinder og en QR-kode kommer til',
    'Bro skal rives ned: Motorvej spærres i 14 timer',
    'Indonesien er klar med Sydøstasiens første højhastighedstog',
    'Politiet dropper efterforskning af hospital og region efter kræftskandale',
    'Hvad foregår der i Ikast? De kom med på et wildcard, og nu topper de hele baduljen'
]

### Lister

Hvis tekster er i en liste, kan man bruge metoden `.pipe()` til at anvende pipeline på flere tekststykker.

`.pipe()` returnerer et "generator object". Dette er en speciel type objekt i Python, som kun giver et output, når den bliver kaldt (en måde at spare hukommelse). Hvis vi fx vil have teksterne tilbage som en liste af `doc`-objekter, kan man tvinge output om til en liste:

In [19]:
docs = list(nlp.pipe(texts))

In [20]:
for word in docs[0]:
    print(f'{word.text:<20} {word.pos_}')

Smileyordningen      NOUN
får                  VERB
stor                 ADJ
makeover             ADP
:                    PUNCT
Kontrolrapporten     NOUN
forsvinder           VERB
og                   CCONJ
en                   DET
QR-kode              NOUN
kommer               VERB
til                  ADP


In [21]:
for word in docs[1]:
    print(f'{word.text:<20} {word.pos_}')

Bro                  ADV
skal                 AUX
rives                VERB
ned                  ADV
:                    PUNCT
Motorvej             PROPN
spærres              VERB
i                    ADP
14                   NUM
timer                NOUN


### Pandas Series

Hvis tekster er i en pandas Series, kan man bruge metoden `.apply()` i pandas. Dog forventer `.apply()`, at der kun gives ét output. Derfor bør man lave en wrapper-funktion, så man kan udpege, hvad der skal gives som output:

In [22]:
# dan funktion til at hente named entities

def get_ents(text):
    doc = nlp(text)

    ents = list(doc.ents)

    return(ents)

In [23]:
import pandas as pd

texts_s = pd.Series(texts) # konvertér liste til series

texts_s.apply(get_ents) # anvend funktion på series

0                       [(QR-kode)]
1                                []
2    [(Indonesien), (Sydøstasiens)]
3                                []
4                         [(Ikast)]
dtype: object